## Heavily adapted from 
[here](https://github.com/pytorch/examples/blob/master/vae/main.py)

In [4]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
batch_size=128
epochs = 1000
log_interval = 100

In [7]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.ToTensor()
                  ),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [14]:
im_dim = 28*28

In [81]:
??nn.MaxUnpool2d()

In [118]:
latent_space_size = 20

In [141]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        #self.conv1 = nn.Conv2d(1, 5, 3, stride=1)
        #self.maxpool1= nn.MaxPool2d(kernel_size=2, return_indices=False)
        #self.fc21 = nn.Linear(845, 20)
        #self.fc22 = nn.Linear(845, 20)
        #self.fc3 = nn.Linear(20, 400)
        #self.unconv1 = nn.ConvTranspose2d(1, 3, 3, stride=1)
        #self.fc4 = nn.Linear(1452, 784)
        
        self.fc1 = nn.Conv2d(1,32, kernel_size=(28,28), stride=1, padding=0)
        self.fc21 = nn.Conv2d(32,latent_space_size, kernel_size=(1,1), stride=1, padding=0)
        self.fc22 = nn.Conv2d(32,latent_space_size, kernel_size=(1,1), stride=1, padding=0)
        
        self.fc3 = nn.ConvTranspose2d(latent_space_size,118, kernel_size=(1,1),  stride=1, padding=0)
        self.fc4 = nn.ConvTranspose2d(118,1, kernel_size=(28,28),stride=1, padding=0)

    #def encode(self, x):
    #    h1 = F.sigmoid(self.maxpool1(self.conv1(x)))
    #    h1 = h1.view(-1, 845)
    #    return self.fc21(h1), self.fc22(h1)

    #def reparameterize(self, mu, logvar):
    #    std = torch.exp(0.5*logvar)
    #    eps = torch.randn_like(std)
    #    return mu + eps*std

    #def decode(self, z):
    #    h3 = F.sigmoid(self.fc3(z)).view(-1,20,20)
    #    h3 = h3.view(-1,1,20,20)
    #    h3 = self.unconv1(h3)
    #    return torch.sigmoid(self.fc4(h3.view(-1,1452)))

    #def forward(self, x):
    #    mu, logvar = self.encode(x)
    #    z = self.reparameterize(mu, logvar)
    #    return self.decode(z), mu, logvar
    
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [142]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [143]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta=1):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + beta * KLD

In [144]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar, beta=2)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [145]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [149]:
## Main
for epoch in range(1, epochs + 1):
    print(epoch)
    train(epoch)
    print("epoch")
    test(epoch)
    print("after")
    #if epochs % 10 == 0:
    #    with torch.no_grad():
    #        sample = torch.randn(64, 20).to(device)
    #        sample = model.decode(sample).cpu()
    #        save_image(sample.view(64, 1, 28, 28),'results/sample_' + str(epoch) + '.png')

1
Train Epoch: 1 [0/60000 (0%)]	Loss: 156.649048


/home/arno/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/arno/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([128, 784])) that is different to the input size (torch.Size([128, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Train Epoch: 1 [12800/60000 (21%)]	Loss: 147.443207
Train Epoch: 1 [25600/60000 (43%)]	Loss: 152.603653
Train Epoch: 1 [38400/60000 (64%)]	Loss: 154.884399
Train Epoch: 1 [51200/60000 (85%)]	Loss: 143.495789


/home/arno/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([96, 784])) that is different to the input size (torch.Size([96, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Epoch: 1 Average loss: 151.6794
epoch


...

====> Test set loss: 129.7117
after
2
Train Epoch: 2 [0/60000 (0%)]	Loss: 146.761383
Train Epoch: 2 [12800/60000 (21%)]	Loss: 152.824844
Train Epoch: 2 [25600/60000 (43%)]	Loss: 145.263184
Train Epoch: 2 [38400/60000 (64%)]	Loss: 152.726562
Train Epoch: 2 [51200/60000 (85%)]	Loss: 145.201035
====> Epoch: 2 Average loss: 147.1863
epoch
====> Test set loss: 125.7795
after
3
Train Epoch: 3 [0/60000 (0%)]	Loss: 151.643967
Train Epoch: 3 [12800/60000 (21%)]	Loss: 141.818085
Train Epoch: 3 [25600/60000 (43%)]	Loss: 149.340500
Train Epoch: 3 [38400/60000 (64%)]	Loss: 142.461304
Train Epoch: 3 [51200/60000 (85%)]	Loss: 139.137726
====> Epoch: 3 Average loss: 144.8802
epoch
====> Test set loss: 124.4922
after
4
Train Epoch: 4 [0/60000 (0%)]	Loss: 148.883179
Train Epoch: 4 [12800/60000 (21%)]	Loss: 140.420898
Train Epoch: 4 [25600/60000 (43%)]	Loss: 145.099411
Train Epoch: 4 [38400/60000 (64%)]	Loss: 148.300629
Train Epoch: 4 [51200/60000 (85%)]	Loss: 143.578033
====> Epoch: 4 Average loss: 143.

====> Test set loss: 115.9900
after
26
Train Epoch: 26 [0/60000 (0%)]	Loss: 126.809418
Train Epoch: 26 [12800/60000 (21%)]	Loss: 134.262085
Train Epoch: 26 [25600/60000 (43%)]	Loss: 136.633102
Train Epoch: 26 [38400/60000 (64%)]	Loss: 136.460587
Train Epoch: 26 [51200/60000 (85%)]	Loss: 136.636139
====> Epoch: 26 Average loss: 135.3937
epoch
====> Test set loss: 116.1227
after
27
Train Epoch: 27 [0/60000 (0%)]	Loss: 131.835388
Train Epoch: 27 [12800/60000 (21%)]	Loss: 137.764221
Train Epoch: 27 [25600/60000 (43%)]	Loss: 133.530304
Train Epoch: 27 [38400/60000 (64%)]	Loss: 134.913651
Train Epoch: 27 [51200/60000 (85%)]	Loss: 135.081665
====> Epoch: 27 Average loss: 135.3669
epoch
====> Test set loss: 115.7214
after
28
Train Epoch: 28 [0/60000 (0%)]	Loss: 140.581436
Train Epoch: 28 [12800/60000 (21%)]	Loss: 131.157562
Train Epoch: 28 [25600/60000 (43%)]	Loss: 133.906357
Train Epoch: 28 [38400/60000 (64%)]	Loss: 134.694809
Train Epoch: 28 [51200/60000 (85%)]	Loss: 136.866302
====> Epoch: 

====> Epoch: 49 Average loss: 134.1010
epoch
====> Test set loss: 114.2990
after
50
Train Epoch: 50 [0/60000 (0%)]	Loss: 134.033478
Train Epoch: 50 [12800/60000 (21%)]	Loss: 140.326965
Train Epoch: 50 [25600/60000 (43%)]	Loss: 130.998093
Train Epoch: 50 [38400/60000 (64%)]	Loss: 135.691895
Train Epoch: 50 [51200/60000 (85%)]	Loss: 132.287262
====> Epoch: 50 Average loss: 134.0529
epoch
====> Test set loss: 114.5496
after
51
Train Epoch: 51 [0/60000 (0%)]	Loss: 138.736893
Train Epoch: 51 [12800/60000 (21%)]	Loss: 134.939514
Train Epoch: 51 [25600/60000 (43%)]	Loss: 141.042755
Train Epoch: 51 [38400/60000 (64%)]	Loss: 130.659500
Train Epoch: 51 [51200/60000 (85%)]	Loss: 138.492325
====> Epoch: 51 Average loss: 134.0569
epoch
====> Test set loss: 114.5586
after
52
Train Epoch: 52 [0/60000 (0%)]	Loss: 132.640945
Train Epoch: 52 [12800/60000 (21%)]	Loss: 131.372894
Train Epoch: 52 [25600/60000 (43%)]	Loss: 132.401764
Train Epoch: 52 [38400/60000 (64%)]	Loss: 138.248260
Train Epoch: 52 [5120

Train Epoch: 73 [51200/60000 (85%)]	Loss: 128.774719
====> Epoch: 73 Average loss: 133.4961
epoch
====> Test set loss: 113.9801
after
74
Train Epoch: 74 [0/60000 (0%)]	Loss: 134.792175
Train Epoch: 74 [12800/60000 (21%)]	Loss: 131.732330
Train Epoch: 74 [25600/60000 (43%)]	Loss: 135.843353
Train Epoch: 74 [38400/60000 (64%)]	Loss: 134.529556
Train Epoch: 74 [51200/60000 (85%)]	Loss: 137.299469
====> Epoch: 74 Average loss: 133.4821
epoch
====> Test set loss: 113.7848
after
75
Train Epoch: 75 [0/60000 (0%)]	Loss: 135.775970
Train Epoch: 75 [12800/60000 (21%)]	Loss: 129.356247
Train Epoch: 75 [25600/60000 (43%)]	Loss: 137.485352
Train Epoch: 75 [38400/60000 (64%)]	Loss: 134.963455
Train Epoch: 75 [51200/60000 (85%)]	Loss: 131.786423
====> Epoch: 75 Average loss: 133.4587
epoch
====> Test set loss: 113.8615
after
76
Train Epoch: 76 [0/60000 (0%)]	Loss: 132.610794
Train Epoch: 76 [12800/60000 (21%)]	Loss: 137.577591
Train Epoch: 76 [25600/60000 (43%)]	Loss: 139.363724
Train Epoch: 76 [3840

KeyboardInterrupt: 

In [96]:
8448 * 22 / 128

1452.0

In [140]:
2560 / 28

91.42857142857143